In [5]:
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from langgraph.checkpoint.memory import InMemorySaver
import psycopg2
from dotenv import load_dotenv
load_dotenv()


True

In [17]:
system_prompt='''You are a PostgreSQL SQL generator.
You have access to exactly ONE table:

Table name: sales_daily

Columns:
- date (DATE) — date of record
- region (TEXT) — region name
- category (TEXT) — product category
- revenue (NUMERIC(12,2)) — revenue amount
- orders (INTEGER) — number of orders
- created_at (TIMESTAMPTZ) — row creation timestamp

Instructions:

1. Generate ONLY a single valid PostgreSQL SELECT query.
2. Do NOT include explanations.
3. Do NOT include markdown formatting.
4. Do NOT include comments.
5. Do NOT include multiple queries.
6. NEVER use INSERT, UPDATE, DELETE, DROP, ALTER, or CREATE.
7. Only query the table: sales_daily.
8. Use correct PostgreSQL syntax.
9. When aggregation is requested, use proper GROUP BY.
10. When filtering by date, use single quotes: 'YYYY-MM-DD'.
11. If the question asks for totals, use SUM().
12. If the question asks for counts, use COUNT().
13. If sorting is implied (e.g., highest, lowest), use ORDER BY.
14. If limiting results makes sense, use LIMIT.

Return only the SQL statement.
If the question cannot be answered using this table, return  short message as to why query is not supported in the format:
SELECT 'Query not supported:{reason query not supported}' AS message;'''

agent=create_agent(
    model="gpt-5-nano",
    system_prompt=system_prompt,
    )


In [18]:
question=HumanMessage(content=[
    {"type":"text","text": "Which elevators have more than 850 countries" }
    ])
response=agent.invoke({"messages":[question]})
agent_query=response['messages'][-1].content
print(response['messages'][-1].content)

SELECT 'Query not supported: The table sales_daily does not contain columns for elevators or countries, so the question cannot be answered with this schema.' AS message;


In [19]:

hostname='localhost'
database='analytics'
username='postgres'
pwd='ss29320#'
port_id=5432
conn=None
cur=None
try:
    conn=psycopg2.connect(
        host=hostname,
        dbname=database,
        user=username,
        password=pwd,
        port=port_id
    )

    cur=conn.cursor()
    # query='SELECT * FROM sales_daily'
    cur.execute(agent_query)
    print(cur.fetchall())
    conn.commit()
except Exception as error:
    print(error)
finally:
    if cur:
        cur.close()
    if conn:
        conn.close()

[('Query not supported: The table sales_daily does not contain columns for elevators or countries, so the question cannot be answered with this schema.',)]
